<a href="https://colab.research.google.com/github/nikhiltree/Labs/blob/master/big_data_pipeline_training_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
print("nikhil")

In [ ]:
!nvidia-smi


In [ ]:
# !pip install colab-xterm
# %load_ext colabxterm
# %xterm

In [ ]:

!pip install wandb
import wandb
api_key="c61cdde2a3147e953ebd7c4d304a16a37a0396ab"
!wandb login --relogin
wandb.init(project="gpu_tracking_feature_tracking")


In [ ]:
!pip install python-dateutil==1.4
!pip install darts
!pip install tslearn

In [ ]:
!python -m pip uninstall matplotlib -y

!pip install  matplotlib==3.1.3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import datetime
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px
import numpy as np
import traceback
from sklearn.preprocessing import MinMaxScaler
from tslearn.svm import TimeSeriesSVR

from darts import TimeSeries
import matplotlib.pyplot as plt
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts import TimeSeries
from darts.utils.timeseries_generation import gaussian_timeseries, linear_timeseries, sine_timeseries
from darts.models import RNNModel, TCNModel, TransformerModel, NBEATSModel, BlockRNNModel
from darts.metrics import mape, smape
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
def acc_measure(y_true=None,y_hat=None):
    total_true = 0
    total_datapoint = len(y_true)
    for i in range(total_datapoint):
        abs_diff = abs((y_true[i]-y_hat[i]))
        y_10 = (y_true[i]/10)
        if abs_diff <= (y_10):
            total_true = total_true+1
    return round((total_true*100/total_datapoint),2)

### File Read

In [ ]:
filename="/content/drive/MyDrive/Colab Notebooks/gmr_big_data/gmr_big_data_area_volume_temperature.csv"

In [ ]:
chunksize = 10 ** 4
list_dfs=[]
with pd.read_csv(filename, chunksize=chunksize) as reader:
    for chunk in reader:
        list_dfs.append(chunk)

In [ ]:
df_ori=pd.concat(list_dfs,axis=0)

In [ ]:
# df_ori=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/region_wise_data/data_preparation_filter_region_volume.csv",thousands=',')

In [ ]:
df_ori.columns

In [ ]:
df_ori.head()

In [ ]:
df_ori['Time Block and Date']=df_ori['Time Block and Date'].astype(str)

In [ ]:
start_date="2021-01-03 00:00:00"
end_date="2021-04-30 00:00:00"

In [ ]:
def select_data_range_wise(df0=None,start_date=None,end_date=None):
  df0['Time Block and Date']=df0['Time Block and Date'].apply(lambda x: x.split(" ")[0]+" "+x.split(" ")[1].split(" ")[0])
  df0['Time Block and Date']=pd.to_datetime(df0["Time Block and Date"],format="%Y-%m-%d %H:%M")
  start_date = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
  end_date = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")
  df0.drop("Time Block",axis=1,inplace=True)
  print(start_date)
  print(end_date)
  target_df0=df0[(df0["Time Block and Date"]<=end_date) & (df0["Time Block and Date"]>=start_date)].reset_index(0,drop=True)
  return target_df0

In [ ]:
df=select_data_range_wise(df0=df_ori.copy(),start_date=start_date,end_date=end_date)

In [ ]:
df.tail()

In [ ]:
df=df.drop(axis=1,labels="MCV_Buy")

In [ ]:
df.info()

In [ ]:
df.head()

## Feature Importance

In [ ]:
def spilting_data_for_xgb(df=None,intial_columns=2):
  X_feature=df.iloc[:,intial_columns:-1]
  y_target=df.iloc[:,-1]
  return X_feature,y_target

In [ ]:
def get_the_parameter_xgb():
  parameters = {#when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.01,0.02,0.03], #so called `eta` value
              'max_depth': [1,2,3,4,5,10],
              'min_child_weight': [30,40,50,100],
              'silent': [0],
              'subsample': [0.4,0.5,0.6,0.7],
              'colsample_bytree': [0.6,0.7,0.8],
              'n_estimators': [5,8,10,20,30,40]
              }
  return parameters

      


In [ ]:
def get_the_parameter_rf():
# Create the parameter grid based on the results of random search 
  param_grid = {
      'bootstrap': [True],
      'max_depth': [80, 90, 100, 110],
      'max_features': [2, 3],
      'min_samples_leaf': [3, 4, 5],
      'min_samples_split': [8, 10, 12],
      'n_estimators': [100, 200, 300, 1000]
  }

  return param_grid

In [ ]:
def get_model_rf():
    rf = RandomForestRegressor()
    return rf

In [ ]:
def get_model():
    xgb1 = XGBRegressor(gpu_id=0)
    return xgb1

In [ ]:
def get_model_parameter_set(parameters=None):
    xgb1 = XGBRegressor(gpu_id=0,parameters=None)
    return xgb1

In [ ]:
def get_model_grid_search(model=None,parameters=None,cv=10):

  xgb_grid = GridSearchCV(model,
                          param_grid=parameters,
                          cv = cv,
                          verbose=2,scoring='neg_mean_absolute_error')
  
  return xgb_grid
  


In [ ]:
def get_best_param(model_grid=None,X_feature=None,y_target=None):

  model_grid.fit(X_feature,y_target)

  # print(model_grid.best_score_) 
  print(model_grid.best_params_)

  return model_grid.best_params_

In [ ]:
def hyperparameter_model_fit(model=None,X_feature=None,y_target=None):
  
  model.fit(X_feature,y_target)

  return model


In [ ]:
def get_importance_feature(model=None,columns=None):
    sorted_idx = model.feature_importances_.argsort()[::-1]
    return columns[sorted_idx][:100]

In [ ]:
def plot_importance_feature(model=None,columns=None):
  plt.figure(figsize=(10,10))
  sorted_idx = model.feature_importances_.argsort()
  plt.barh(columns[sorted_idx], model.feature_importances_[sorted_idx])
  plt.xlabel("Xgboost Feature Importance")

## based model training

In [ ]:
X_features,y_train=spilting_data_for_xgb(df=df)
xgb_model=get_model()
hyperpameter=get_the_parameter_xgb()

grid_model=get_model_grid_search(model=xgb_model,parameters=hyperpameter)
best_params=get_best_param(grid_model,X_features,y_train)

model_best=get_model_parameter_set(parameters=best_params)
model_best=hyperparameter_model_fit(model=model_best,X_feature=X_features,y_target=y_train)

best_feature_list=get_importance_feature(model=model_best,columns=X_features.columns)

plot_importance_feature(model=model_best,columns=X_features.columns)

In [ ]:
best_params

In [ ]:
best_feature_list=best_feature_list.to_list()



In [ ]:
best_feature_list=['MCP (Rs/MWh) ', 'W2_Sell', 'W1_Sell']

In [ ]:
# best_params={'colsample_bytree': 0.7,
#  'learning_rate': 0.05,
#  'max_depth': 6,
#  'min_child_weight': 4,
#  'n_estimators': 120,
#  'objective': 'reg:linear',
#  'silent': 1,
#  'subsample': 0.7}

## date extraction and conversion object to date 

In [ ]:
df.info()

In [ ]:

df.columns

### Columns selection for training

In [ ]:
columns=["Time Block and Date"]+best_feature_list+["Target MCP"]

In [ ]:
def columns_selection_df(df=None,columns_list=None):
  df1=df[columns]
  return df1

In [ ]:
df1=df[columns]

In [ ]:
df1

In [ ]:
df1.columns

In [ ]:
df1.head()

In [ ]:
df1.info()

## Training and Testing Data Generation



### Method 1: Training Data Generation split(Example: Train 0.80 and Test 0.20)

In [ ]:
def get_index_test_date(df,test_size=None):
  date_list=df.iloc[:,0]
  date_list=date_list.tail(int(len(df)*(test_size)))
  return date_list

In [ ]:
def default_training_test_generation(first_skip_columns=1,df0=None):
  X=df0.iloc[:,first_skip_columns:-1]
  y=df0.iloc[:,-1]
  return X,y

def train_split_data(X=None,y=None,test_size=None):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0,shuffle=False)
  return X_train,X_test,y_train,y_test
   

### Method 2: Select test date  and pervious all day training

In [ ]:
def selection_test_date_data_generation(df0=None,test_date=None):
  training_last_date=test_date-timedelta(days=1)
  print(training_last_date)
  temp_df=df0[df0['Time Block and Date']<=test_date].reset_index(0,drop=True)
  train_df=temp_df[temp_df['Time Block and Date']<=training_last_date].reset_index(0,drop=True)
  test_df=temp_df[temp_df['Time Block and Date']>training_last_date].reset_index(0,drop=True)

  return train_df,test_df
 


In [ ]:
def selection_test_date_train_test_split(train_df=None,test_df=None,first_skip_columns=1):
  X_train,y_train=train_df.iloc[:,first_skip_columns:-1],train_df.iloc[:,-1]
  X_test,y_test=test_df.iloc[:,first_skip_columns:-1],test_df.iloc[:,-1]
  return X_train,X_test,y_train,y_test


### Method 3: select training date interval  and test_date interval

In [ ]:
def selection_interval_train_test_data_generation(df=None,start_train_date=None,end_train_date=None,start_test_date=None,end_test_date=None):

  train_df=df[(df['Time Block and Date']>=start_train_date) & (df['Time Block and Date']<end_train_date)].reset_index(0,drop=True)
  print(end_test_date)
  test_df=df[(df['Time Block and Date']>=start_test_date) & (df['Time Block and Date']<end_test_date)].reset_index(0,drop=True)


  return train_df,test_df
 

In [ ]:
def selection_interval_train_test_split(train_df=None,test_df=None,first_skip_columns=1):
  X_train,y_train=train_df.iloc[:,first_skip_columns:-1],train_df.iloc[:,-1]
  X_test,y_test=test_df.iloc[:,first_skip_columns:-1],test_df.iloc[:,-1]
  return X_train,X_test,y_train,y_test

In [ ]:
start_training_date=datetime(2022,10,29,0,0)
end_training_date=datetime(2022,11,4,0,0)
start_testing_date=datetime(2022,11,4,0,0)
end_testing_date=datetime(2022,11,5,0,0)


In [ ]:
train_df_interval,test_df_interval=selection_interval_train_test_data_generation(df=df1,start_train_date=start_training_date,end_train_date=end_training_date,start_test_date=start_testing_date,end_test_date=end_testing_date)

In [ ]:
train_df_interval

In [ ]:
test_df_interval

## Trained and Test Model With Method 1:

In [ ]:
df1

In [ ]:
X,y=default_training_test_generation(df0=df1)
X_train,X_test,y_train,y_test=train_split_data(X=X,y=y,test_size=0.09)

In [ ]:
test_date_list=get_index_test_date(df1,0.091).to_list()

In [ ]:
len(test_date_list)

In [ ]:
len(y_test)


#### Hyperparameter Find

In [ ]:
xgb_model=get_model()
hyperpameter=get_the_parameter_xgb()

grid_model=get_model_grid_search(model=xgb_model,parameters=hyperpameter)
best_params=get_best_param(grid_model,X_train,y_train)

model_best=get_model_parameter_set(parameters=best_params)
model_best=hyperparameter_model_fit(model=model_best,X_feature=X_train,y_target=y_train)

best_feature_list=get_importance_feature(model=model_best,columns=X_train.columns)

plot_importance_feature(model=model_best,columns=X_train.columns)

In [ ]:
y_pred_m1=model_best.predict(X_test)

In [ ]:
len(y_pred_m1)

In [ ]:
acc_measure(y_true=y_test.to_numpy(),y_hat=y_pred_m1)

## Trained and Test Model With Method 2:

In [ ]:
test_time=datetime(2022,11,2,23,45)

In [ ]:
train_df,test_df=selection_test_date_data_generation(df0=df1,test_date=test_time)

In [ ]:
X_train,X_test,y_train,y_test=selection_test_date_train_test_split(train_df=train_df,test_df=test_df)

In [ ]:
X_train

#### Hyperparmeter M2

  #### Xgboost




In [ ]:
xgb_model=get_model()
hyperpameter=get_the_parameter_xgb()


#### Random Forest


In [ ]:
# rf_model=get_model_rf()
# hyperpameter_rf=get_the_parameter_rf()

In [ ]:
grid_model=get_model_grid_search(model=xgb_model,parameters=hyperpameter)
best_params=get_best_param(grid_model,X_train,y_train)

model_best=get_model_parameter_set(parameters=best_params)
model_best=hyperparameter_model_fit(model=model_best,X_feature=X_train,y_target=y_train)

best_feature_list=get_importance_feature(model=model_best,columns=X_train.columns)

plot_importance_feature(model=model_best,columns=X_train.columns)

In [ ]:
y_pred_m2=model_best.predict(X_test)

In [ ]:
y_pred_m2

In [ ]:
acc_measure(y_true=y_test.to_numpy(),y_hat=y_pred_m2)

## Trained and Test Model With Method 3:

In [ ]:
start_training_date=datetime(2022,10,30,0,0)
end_training_date=datetime(2022,11,4,0,0)
start_testing_date=datetime(2022,11,10,0,0)
end_testing_date=datetime(2022,11,11,0,0)


In [ ]:
train_df_interval,test_df_interval=selection_interval_train_test_data_generation(df=df1,start_train_date=start_training_date,end_train_date=end_training_date,start_test_date=start_testing_date,end_test_date=end_testing_date)

In [ ]:
train_df_interval

In [ ]:
test_df_interval

In [ ]:
X_train,X_test,y_train,y_test=selection_interval_train_test_split(train_df=train_df_interval,test_df=test_df_interval)

In [ ]:
xgb_model=get_model()
hyperpameter=get_the_parameter_xgb()
grid_model=get_model_grid_search(model=xgb_model,parameters=hyperpameter)
best_params=get_best_param(grid_model,X_train,y_train)

model_best=get_model_parameter_set(parameters=best_params)
model_best=hyperparameter_model_fit(model=model_best,X_feature=X_train,y_target=y_train)


In [ ]:
y_pred_m3=model_best.predict(X_test)
acc_measure(y_true=y_test.to_numpy(),y_hat=y_pred_m3)

## scroling window size training and test

In [ ]:
import dateutil
from datetime import date,timedelta

In [ ]:
def chunk_training_test_date_generator(list_dates=[],window_size=3):
  scroling_window_date_list=[]
  for i in range(len(list_dates) - window_size + 1):
      scroling_window_date_list.append(list_dates[i: i + window_size])
  return scroling_window_date_list
  


In [ ]:
def generate_all_dates(df0=None):
    date_list_dataframe = list(
    dateutil.rrule.rrule(dateutil.rrule.DAILY, 
                         dtstart=df0['Time Block and Date'][0],
                         until=df0['Time Block and Date'].iloc[-1]))
    return date_list_dataframe


In [ ]:
def get_scorling_window_accuracy(df0,window_size=3):
    list_all_dates=generate_all_dates(df0=df0)
    chunk_list=chunk_training_test_date_generator(list_dates=list_all_dates,window_size=window_size)
    testing_date_list=[]
    list_testing_acc=[]
    list_training_acc=[]
    print("window_size:-",window_size)
    for chunk in chunk_list:
        print(chunk)
        testing_date_list.append(chunk[-1])
        start_training_date=chunk[0]
        end_training_date=chunk[-1]
        start_testing_date=chunk[-1]
        end_testing_date=chunk[-1]+timedelta(days=1)

      
        train_df_interval,test_df_interval=selection_interval_train_test_data_generation(df=df1,start_train_date=start_training_date,end_train_date=end_training_date,start_test_date=start_testing_date,end_test_date=end_testing_date)
        X_train,X_test,y_train,y_test=selection_interval_train_test_split(train_df=train_df_interval,test_df=test_df_interval)
        xgb_model=get_model()
        hyperpameter=get_the_parameter_xgb()
        grid_model=get_model_grid_search(model=xgb_model,parameters=hyperpameter)
        best_params=get_best_param(grid_model,X_train,y_train)
        

        model_best=get_model_parameter_set(parameters=best_params)
        model_best=hyperparameter_model_fit(model=model_best,X_feature=X_train,y_target=y_train)
        y_pred_test_scroling_window=model_best.predict(X_test)
        y_pred_train_scroling_window=model_best.predict(X_train)

        acc_test=acc_measure(y_true=y_test.to_numpy(),y_hat=y_pred_test_scroling_window)
        acc_train=acc_measure(y_true=y_train.to_numpy(),y_hat=y_pred_train_scroling_window)

        list_testing_acc.append(acc_test)
        list_training_acc.append(acc_train)
        print("Best_parameter",best_params)
    

        
        
    return {"testing_dates":testing_date_list,"testing_acc":list_testing_acc,"training_acc":list_training_acc}


In [ ]:
result_df=get_scorling_window_accuracy(df0=df1,window_size=20)

In [ ]:
result_df

In [ ]:
result_df_form=pd.DataFrame(result_df)

In [ ]:
result_df_form

In [ ]:
result_df_form.describe()

## Experment Code start(Trash)




### experiment xgboost function training and testing generalize

## dart experiment start

#### filter df by using date we want to predict

In [ ]:
df1.columns

In [ ]:
df1['Time Block and Date'].unique()

In [ ]:
target_date="2022-11-10 00:00:00"

In [ ]:
def get_target_df(df1,target_date):
  datetime_str = datetime.datetime.strptime(target_date, "%Y-%m-%d %H:%M:%S")
  target_df=df1[df1["Time Block and Date"]<=datetime_str]
  return target_df

In [ ]:
def target_date_format(datetime_str=None):
  datetime_str = datetime.datetime.strptime(target_date, "%Y-%m-%d %H:%M:%S")
  return datetime_str

In [ ]:
datetime_str = datetime.datetime.strptime(target_date, "%Y-%m-%d %H:%M:%S")

In [ ]:
datetime_str

In [ ]:

df1=df1[df1["Time Block and Date"]<datetime_str]

In [ ]:
df1

## convert dataframe to time series


In [ ]:
def convert_df_to_train_test(df=None):
  '''split data into last test date and training date'''
  X_train,X_test=df1.iloc[:-96,:-1],df1.iloc[-96:,:-1]
  y_train,y_test=df1.iloc[:-96,[0,-1]],df1.iloc[-96:,[0,-1]]

  return X_train,X_test,y_train,y_test
  

In [ ]:
def convert_train_test_to_time_series(_x_train=None,x_test=None,y_train=None,y_test=None):
  X_train=TimeSeries.from_dataframe(X_train,time_col="Time Block and Date",value_cols=X_train.columns[1:].to_list(),fill_missing_dates=True)
  X_test=TimeSeries.from_dataframe(X_test,time_col="Time Block and Date",value_cols=X_test.columns[1:].to_list(),fill_missing_dates=True)
  y_train=TimeSeries.from_dataframe(y_train,time_col="Time Block and Date",value_cols=y_train.columns[1],fill_missing_dates=True)
  y_test=TimeSeries.from_dataframe(y_test,time_col="Time Block and Date",value_cols=y_test.columns[1],fill_missing_dates=True)
  return X_train,X_test,y_train,y_test

In [ ]:
validation_day=5

In [ ]:
window=96

## validation cov_t prepare

In [ ]:
cov_t=TimeSeries.from_dataframe(df1,time_col="Time Block and Date",value_cols=df1.columns[1:-1].to_list(),fill_missing_dates=True)

In [ ]:
cov_t_m = datetime_attribute_timeseries(cov_t.time_index, attribute="minute")   # 48 hours beyond end of test set to prepare for out-of-sample forecasting
cov_t_w = datetime_attribute_timeseries(cov_t.time_index, attribute="weekday")

In [ ]:
cov_t=cov_t.concatenate(cov_t_m,axis=1)
cov_t=cov_t.concatenate(cov_t_w,axis=1)


In [ ]:
cov_t.columns

In [ ]:
X_train,X_test=df1.iloc[:validation_day*(-window),:-1],df1.iloc[validation_day*(-window):,:-1]
y_train,y_test=df1.iloc[:validation_day*(-window),[0,-1]],df1.iloc[validation_day*(-window):,[0,-1]]

In [ ]:
# X_train,X_test=df1.iloc[:-96,:-1],df1.iloc[-96:,:-1]
# y_train,y_test=df1.iloc[:-96,[0,-1]],df1.iloc[-96:,[0,-1]]

In [ ]:
X_train.columns

In [ ]:
X_train=TimeSeries.from_dataframe(X_train,time_col="Time Block and Date",value_cols=X_train.columns[1:].to_list(),fill_missing_dates=True)
X_test=TimeSeries.from_dataframe(X_test,time_col="Time Block and Date",value_cols=X_test.columns[1:].to_list(),fill_missing_dates=True)


In [ ]:
y_train=TimeSeries.from_dataframe(y_train,time_col="Time Block and Date",value_cols=y_train.columns[1],fill_missing_dates=True)
y_test=TimeSeries.from_dataframe(y_test,time_col="Time Block and Date",value_cols=y_test.columns[1],fill_missing_dates=True)


### Train Covariates

In [ ]:
def convert_time_series_data_to_X_covariates(X_train=None,X_test=None):
  covT_train = datetime_attribute_timeseries(X_train.time_index, attribute="minute")   # 48 hours beyond end of test set to prepare for out-of-sample forecasting
  covT_train = covT_train.stack(  datetime_attribute_timeseries(covT_train.time_index, attribute="weekday"))
  covT_train=covT_train.concatenate(X_train,axis=1)
  covT_test = datetime_attribute_timeseries(X_test.time_index, attribute="minute")   # 48 hours beyond end of test set to prepare for out-of-sample forecasting
  covT_test = covT_test.stack(  datetime_attribute_timeseries(covT_test.time_index, attribute="weekday"))
  covT_test=covT_test.concatenate(X_train,axis=1)
  return covT_train,covT_test
  


In [ ]:
covT_train = datetime_attribute_timeseries(X_train.time_index, attribute="minute")   # 48 hours beyond end of test set to prepare for out-of-sample forecasting
covT_train = covT_train.stack(  datetime_attribute_timeseries(covT_train.time_index, attribute="weekday"))
# covT_train = covT_train.stack(  datetime_attribute_timeseries(covT_train.time_index, attribute="month")  )
# covT_train = covT_train.stack(  datetime_attribute_timeseries(covT_train.time_index, attribute="year")  )

In [ ]:
covT_train=covT_train.concatenate(X_train,axis=1)

### Test Covarites

In [ ]:
covT_test = datetime_attribute_timeseries(X_test.time_index, attribute="minute")   # 48 hours beyond end of test set to prepare for out-of-sample forecasting
# covT_test = covT_test.stack(  datetime_attribute_timeseries(covT_test.time_index, attribute="day_of_week")  )
# covT_test = covT_test.stack(  datetime_attribute_timeseries(covT_test.time_index, attribute="month")  )
# covT_test = covT_test.stack(  datetime_attribute_timeseries(covT_test.time_index, attribute="year")  )

In [ ]:
covT_test=covT_test.concatenate(X_test,axis=1)

## y_train Covariate

In [ ]:
y_test

In [ ]:
# def convert_time_series_data_to_Y_covariates(y_train=None,y_test=None):
#   covT_y_train = datetime_attribute_timeseries( y_train.time_index, attribute="minute")

In [ ]:
covT_y_train = datetime_attribute_timeseries( y_train.time_index, attribute="minute")   # 48 hours beyond end of y_train set to prepare for out-of-sample forecasting
# covT_y_train = covT_y_train.stack(  datetime_attribute_timeseries(covT_y_train.time_index, attribute="weekday")  )
# covT_y_train = covT_y_train.stack(  datetime_attribute_timeseries(covT_y_train.time_index, attribute="month")  )
# covT_y_train = covT_y_train.stack(  datetime_attribute_timeseries(covT_y_train.time_index, attribute="year")  )

In [ ]:
y_train

## scaling

In [ ]:
# from darts.dataprocessing.transformers import Scaler
# X_train_scaler, y_train_scaler = Scaler(), Scaler()
# covT_train_scaled = X_train_scaler.fit_transform(covT_train)
# y_train_scaled = y_train_scaler.fit_transform(covT_y_train)

## Custom Metrice Design 

In [ ]:
!pip install pytorch-lightning

In [ ]:
import pytorch_lightning

In [ ]:
!pip install torchmetrics

In [ ]:
# from torchmetrics import Metric

# class MyAccuracy(Metric):
#     def __init__(self):
#         super().__init__()
#         self.add_state("correct", default=torch.tensor(0), dist_reduce_fx="sum")
#         self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")

#     def update(self, preds: torch.Tensor, target: torch.Tensor):
#         preds, target = self._input_format(preds, target)
#         assert preds.shape == target.shape
        
        
#         self.correct = torch.abs(target-preds)
#         self.correct=self.correct[self.correct<10].size()
#         self.total = target.size()

#     def compute(self):
#         return self.correct.float() / self.total.float()

In [ ]:
# import torch 

In [ ]:
# def acc_custom_metric(y_predictions, target):
#   abs_val = torch.abs(y_predictions - target)
#   abs_val=abs_val[abs_val<10]
#   return len(abs_val)


## model training

In [ ]:
covT_train

In [ ]:
!nvidia-smi

In [ ]:
covT_test

In [ ]:
from torchmetrics import MeanAbsolutePercentageError

### model fitting 

In [ ]:
model_mcp = NBEATSModel(input_chunk_length=96*4, output_chunk_length=96, n_epochs=4, random_state=0,force_reset=True,batch_size=16,
    pl_trainer_kwargs={
      "accelerator": "gpu",
      "devices": [0]
    },torch_metrics= MeanAbsolutePercentageError(),
    model_name="NBEATS_EnergyIND",
    log_tensorboard=True,
    generic_architecture=True, 
    save_checkpoints=True)
model_mcp.fit(series=y_train,past_covariates=cov_t,val_series=y_test,
                val_past_covariates=cov_t,verbose=True)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/darts_logs/NBEATS_EnergyES/logs


In [ ]:
pred = model_mcp.predict(n=96,  past_covariates=covT_train)


In [ ]:
model_mcp.batch_norm

In [ ]:
dir(model_mcp)

In [ ]:
model_mcp.model_params

In [ ]:
import shutil
shutil.make_archive("dart_log", 'zip', "/content/darts_logs")

In [ ]:
pred

In [ ]:
plt.figure(figsize=(10,10))
pred = model_mcp.predict(n=96*5)
pred.plot(label='forecast')
plt.legend();


In [ ]:
plt.figure(figsize=(50,10))
y_test.plot(label='real')
pred.plot(label='forecast')
plt.legend();

In [ ]:

y_hat=pred.values()

In [ ]:
y_test_true=y_test.values()

In [ ]:
pred

In [ ]:
acc_measure(y_hat=list(y_hat),y_true=list(y_test_true))

## trash code

In [ ]:
covT_test

#### Time block wise data selection

In [ ]:
df1.columns

In [ ]:
time_block_list=df1['Time Block'].unique()
time_block_list